In [2]:
if (!require(survRM2)) install.packages('survRM2')
library('survRM2') 
if (!require(mvtnorm)) install.packages('mvtnorm')
library('mvtnorm')
if (!require(survival)) install.packages('survival')
library('survival')
if (!require(nph)) install.packages('nph')
library('nph')
if (!require(foreach)) install.packages("foreach")
library('foreach') 
if (!require(doParallel)) install.packages("doParallel")
library('doParallel')
if (!require(ggplot2)) install.packages("ggplot2")
library('ggplot2')
if (!require(cowplot)) install.packages("cowplot")
library('cowplot')

n_cores <- detectCores()
cluster <- makeCluster(14) # How many cores we use
registerDoParallel(cluster)
# multi thread
invisible(clusterEvalQ(cluster, #import packages to parallel 
  {
  library('survRM2')
  library('mvtnorm')
  library('survival')
  library('nph')
  library("simtrial")
  library("foreach")
  }))

source('/home/r27user6/RMST_Code/Function.R')
clusterExport(cluster, "expo_gen_2stages")

# Use the RMST difference test in 2 stages design
#### Rejection region of Interim:
#### $\textcolor{lightgreen}{E(\tau_1) > C(\tau_1)}$
#### Rejection region of overall:
#### $ \textcolor{lightgreen}{E(\tau_1) > C(\tau_1)\ \&\  E(\tau_2)>C(\tau_2) }$

#### Similar to Shan(2021) (https://doi.org/10.1016/j.conctc.2021.100732)
#### But this paper is single arm. We are double
#### We use the non-HR setting in Eaton(2020)
####  <span style="color:yellow">Early differenct: A hazard ratio of 0.67 until 15 months and then a hazard ratio of 1.2 

In [ ]:
median_con <- 10 # month
lambda_H0 <- log(2)/median_con
HR1 <- 0.67
HR2 <- 1.2
sim_size <- 10000 
acc_time <- 24
cen_time <- 36
n <- 100  # Fix total sample size
interim <- 20
change_time <- 15
set.seed(2024)

data_C <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0,
                         dist = 'exp', cen_time = cen_time, arm = 0, interim = interim)
data_E_H0 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, 
                         dist = 'exp', cen_time = cen_time, arm = 1, interim = interim)                           
data_E_H1 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, 
                         dist = 'pcw_exp', cen_time = cen_time,HR1 = HR1, HR2 = HR2, 
                         change_time = change_time, arm = 1, interim = interim)

#### Stage I (Interim) type I error can be set and $PET_0=1-\alpha_1 $
#### Under each $\alpha_1$ we can get empirical $power_1 = 1-PET_1$


In [ ]:
power_1 <- RMST_sim_test(n = 100, data_C = data_C, data_E = data_E_H1, tau = 48, 
                        sim_size = sim_size, alpha = 0.1, sided = 'greater')$test_result 